#### Open-API 방식

- 특정 기관에서 데이터를 제공하는 방식
- 웹 URL을 이용해서 제공하는 방식(정확하게는 웹프로토콜을 이용해서 제공)
    - 프로토콜 : 규약, 규칙이라는 의미
- 데이터를 요청하는 사람(수집)과 데이터를 제공하는 기관이 있음
- 수집자는 제공자가 제공하는 URL을 사용해야 함
    - 제공자가 정의한 key : value 값들을 받아와서 사용해야 함
- 제공자는 아무에게나 데이터를 제공하지 않으며 승인된 수집자에게만 데이터를 제공
    - 승인여부 : 제공자가 발급한 API-Key를 수집자가 전달받아서 요청 시 사용

- 중요 용어
    - 수집자 : 요청자(클라이언트, client) == request
    - 제공자 : 응답자(서버, server) == response

In [1]:
# 공공데이터포탈 접속 : www.data.go.kr
# "한국공항공사 전국공항 주차장 혼잡도" 검색
# [활용신청]

#### 사용할 라이브러리 정의

In [2]:
### 데이터 처리
import pandas as pd

### 웹 프로토콜을 통해서 요청 URL을 전송하고 결과를 받아오는 라이브러리
import requests

### 전달받은 태그 코드의 데이터를 검색 추출하기 위한 라이브러리
from bs4 import BeautifulSoup

### 전달받은 태그 코드를 인식하지 못하는 경우를 대비해서 라이브러리 추가
# 설치 필요 : pip install Lxml
# 설치 후 커널 재시작 필요
from openpyxl.workbook import workbook

#### 요청 처리를 위한 변수 정의하기

In [3]:
### API 인증키 변수 정의
# api_key = "E%2FDZS%2BYqnpd9f0TfrD2raWGtK6X1h2YNmw1Zl4mPcmQBGG9frAt6TFBVfIjCz5uqPd6q%2BVUIMrjkhvwKbeSuUw%3D%3D"
api_key = "PVdeh44WlF%2BHj6bGzfI2BXUldlea7Ggo%2FjjXvhbHCMNNFotchb2byGTopt4uAFS07RQzqmeU3M2Y8HcMmlYW9w%3D%3D"

### 요청에 사용할 공항코드 변수 정의
schAirportCode_list = [ "GMP", "PUS", "CJU", "TAE", "KWJ", "RSU", "USN", "KUV", "WJU", "CJJ" ]

### 한페이지 행의 수
numOfRows = 10

### 페이지 번호
pageNo = 1

In [4]:
### 요청 URL 완성하기
api_url = "http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT"
api_url += f"?schAirportCode={schAirportCode_list[0]}&serviceKey={api_key}&numOfRows={numOfRows}&pageNo={pageNo}"

api_url

'http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT?schAirportCode=GMP&serviceKey=PVdeh44WlF%2BHj6bGzfI2BXUldlea7Ggo%2FjjXvhbHCMNNFotchb2byGTopt4uAFS07RQzqmeU3M2Y8HcMmlYW9w%3D%3D&numOfRows=10&pageNo=1'

#### 요청 및 응답 받아오기

In [5]:
### requests 라이브러리를 이용해서 서버로 url을 넘겨주기(요청)
rs_data = requests.get(api_url)

In [6]:
### 응답 받은 데이터 확인하기
rs_text = rs_data.text

In [7]:
### 문자열 타입의 데이터에 태그(xml) 포멧이라는 의미 부여하기
# - 파싱(변환, parsing) 처리라고 함
rs_xml = BeautifulSoup(rs_text, "xml")

rs_xml

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제1주차장</parkingAirportCodeName><parkingCongestion>혼잡</parkingCongestion><parkingCongestionDegree>91.14%</parkingCongestionDegree><parkingOccupiedSpace>2077</parkingOccupiedSpace><parkingTotalSpace>2279</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>10:13:03</sysGettime></item><item><airportEng>GIMPO INTERNATIONAL AIRPORT</airportEng><airportKor>김포국제공항</airportKor><parkingAirportCodeName>국내선 제2주차장</parkingAirportCodeName><parkingCongestion>원활</parkingCongestion><parkingCongestionDegree>51.7%</parkingCongestionDegree><parkingOccupiedSpace>896</parkingOccupiedSpace><parkingTotalSpace>1733</parkingTotalSpace><sysGetdate>2025-02-21</sysGetdate><sysGettime>10:13:03</sysGettime></item><item><airportEng>GIMPO INTERNATIO

In [8]:
### 태그 이름 : key
### 태그와 태그 사이의 텍스트 : value
# - find_all(찾고자하는 태그이름-key) : 해당 태그 이름이 있는 모든 태그를 리스트 타입으로 반환
rs_xml.find_all("airportKor")

[<airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>,
 <airportKor>김포국제공항</airportKor>]

In [9]:
# - find(찾고자하는 태그이름-key) : 해당 태그 이름이 있는 모든 태그중에서 첫번째 태그만 반환(xml 타입)
rs_xml.find("airportKor")

<airportKor>김포국제공항</airportKor>

In [10]:
### 태그와 태그 사이의 값 추출하기
# - xml 타입인 경우만 가능
# - 문자열로 반환함
rs_xml.find("airportKor").text

'김포국제공항'

In [11]:
### fine_all()을 사용할 땐 순서 선택
rs_xml.find_all("airportKor")[0].text

'김포국제공항'

In [12]:
### 주차장 혼잡률에 대한 모든 태그 정보 가져오기
# - parkingCongestionDegree : 주차장 혼잡률

rs_xml.find_all("parkingCongestionDegree")

[<parkingCongestionDegree>91.14%</parkingCongestionDegree>,
 <parkingCongestionDegree>51.7%</parkingCongestionDegree>,
 <parkingCongestionDegree>89.77%</parkingCongestionDegree>,
 <parkingCongestionDegree>86.48%</parkingCongestionDegree>,
 <parkingCongestionDegree>100%</parkingCongestionDegree>]

In [13]:
### 리스트 중 0번째에 대해서만

### 혼잡도
print(rs_xml.find_all("parkingCongestion")[0].text),

### 주차장 이름
print(rs_xml.find_all("parkingAirportCodeName")[0].text),

### 총주차면수
print(rs_xml.find_all("parkingTotalSpace")[0].text),

### 입고된차량수
print(rs_xml.find_all("parkingOccupiedSpace")[0].text),

### 수집 기준년월일
print(rs_xml.find_all("sysGetdate")[0].text),

### 수집 기준시간
print(rs_xml.find_all("sysGettime")[0].text),

### 주차 가능한 여유공간 수
# 전체 - 입고된 주차
temp = int(rs_xml.find_all("parkingTotalSpace")[0].text) - int(rs_xml.find_all("parkingOccupiedSpace")[0].text)
print(temp)


혼잡
국내선 제1주차장
2279
2077
2025-02-21
10:13:03
202


In [14]:
### 김포 국제공항에 대한 전체 주차장 정보 출력

"""
출력결과
[공항명] : 김포국제공항
[기준 년월일 시분초] : 0000-00-00 00:00:00
-**- -**- -**- -**- -**- -**- -**- -**- -**-
[주차장명] : 국내선 제 1주차장
    총주차면수 : 2279
    ....
    주차장 정보 모두
-**- -**- -**- -**- -**- -**- -**- -**- -**-
[주차장명] : 국내선 제 2주차장
    총주차면수 : 2279
    ....
    주차장 정보 모두
"""

'\n출력결과\n[공항명] : 김포국제공항\n[기준 년월일 시분초] : 0000-00-00 00:00:00\n-**- -**- -**- -**- -**- -**- -**- -**- -**-\n[주차장명] : 국내선 제 1주차장\n    총주차면수 : 2279\n    ....\n    주차장 정보 모두\n-**- -**- -**- -**- -**- -**- -**- -**- -**-\n[주차장명] : 국내선 제 2주차장\n    총주차면수 : 2279\n    ....\n    주차장 정보 모두\n'

In [15]:
len(rs_xml.find_all("airportKor"))

5

In [16]:
## 출력
print("")
print(f"[공항명] : {rs_xml.find_all("airportKor")[0].text}")
print(f"[기준년월시] : {rs_xml.find_all("sysGetdate")[0].text} {rs_xml.find_all("sysGettime")[0].text}")
print("")
for idx in range(len(rs_xml.find_all("airportKor"))) :
    print("-**- -**- -**- -**- -**- -**- -**- -**- -**-")
    print("")
    print(f"[주차장명] : {rs_xml.find_all("parkingAirportCodeName")[idx].text}")
    print(f"    - [총주차면수] : {rs_xml.find_all("parkingTotalSpace")[idx].text}")
    print(f"    - [주차장 혼잡정도] : {rs_xml.find_all("parkingCongestion")[idx].text}")
    print(f"    - [주차장 혼잡률] : {rs_xml.find_all("parkingCongestionDegree")[idx].text}")
    print(f"    - [입고된 차량 수] : {rs_xml.find_all("parkingOccupiedSpace")[idx].text}")
    print(f"    - [전체 주차면 수] : {rs_xml.find_all("parkingTotalSpace")[idx].text}")
    temp = int(rs_xml.find_all("parkingTotalSpace")[idx].text) - int(rs_xml.find_all("parkingOccupiedSpace")[idx].text)
    print(f"    - [주차 가능 여유공간] : {temp}")
    print("")


[공항명] : 김포국제공항
[기준년월시] : 2025-02-21 10:13:03

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국내선 제1주차장
    - [총주차면수] : 2279
    - [주차장 혼잡정도] : 혼잡
    - [주차장 혼잡률] : 91.14%
    - [입고된 차량 수] : 2077
    - [전체 주차면 수] : 2279
    - [주차 가능 여유공간] : 202

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국내선 제2주차장
    - [총주차면수] : 1733
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 51.7%
    - [입고된 차량 수] : 896
    - [전체 주차면 수] : 1733
    - [주차 가능 여유공간] : 837

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국제선 주차빌딩
    - [총주차면수] : 567
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 89.77%
    - [입고된 차량 수] : 509
    - [전체 주차면 수] : 567
    - [주차 가능 여유공간] : 58

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국제선 지하
    - [총주차면수] : 599
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 86.48%
    - [입고된 차량 수] : 518
    - [전체 주차면 수] : 599
    - [주차 가능 여유공간] : 81

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 화물청사
    - [총주차면수] : 737
    - [주차장 혼잡정도] : 만차
    - [주차장 혼잡률] : 100%
    - [입고된 차량 

#### 전체 국내 항공에 대한 주차장 정보 조회하기

In [17]:
### 전체 국내 공항 코드
schAirportCode_list

['GMP', 'PUS', 'CJU', 'TAE', 'KWJ', 'RSU', 'USN', 'KUV', 'WJU', 'CJJ']

In [18]:
### 요청 URL 완성하기

for num in range(len(schAirportCode_list)) :
    api_url = "http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT"
    api_url += f"?schAirportCode={schAirportCode_list[num]}&serviceKey={api_key}&numOfRows={numOfRows}&pageNo={pageNo}"
    rs_data = requests.get(api_url)
    rs_text = rs_data.text
    rs_xml = BeautifulSoup(rs_text, "xml")
    print(f"[공항명] : {rs_xml.find("airportKor").text}")


[공항명] : 김포국제공항
[공항명] : 김해국제공항
[공항명] : 제주국제공항
[공항명] : 대구국제공항
[공항명] : 광주공항
[공항명] : 여수공항
[공항명] : 울산공항
[공항명] : 군산공항
[공항명] : 원주공항
[공항명] : 청주국제공항


In [19]:
for num in range(len(schAirportCode_list)) :
    api_url = "http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT"
    api_url += f"?schAirportCode={schAirportCode_list[num]}&serviceKey={api_key}&numOfRows={numOfRows}&pageNo={pageNo}"
    rs_data = requests.get(api_url)
    rs_text = rs_data.text
    rs_xml = BeautifulSoup(rs_text, "xml")
    print("-**--******- -******- -******- -******- -**-")
    print("")
    print(f"[공항명] : {rs_xml.find("airportKor").text}")
    print(f"[기준년월시] : {rs_xml.find("sysGetdate").text} {rs_xml.find("sysGettime").text}")
    print("")
    for idx in range(len(rs_xml.find_all("airportKor"))) :
        print("-**- -**- -**- -**- -**- -**- -**- -**- -**-")
        print("")
        print(f"[주차장명] : {rs_xml.find_all("parkingAirportCodeName")[idx].text}")
        print(f"    - [총주차면수] : {rs_xml.find_all("parkingTotalSpace")[idx].text}")
        print(f"    - [주차장 혼잡정도] : {rs_xml.find_all("parkingCongestion")[idx].text}")
        print(f"    - [주차장 혼잡률] : {rs_xml.find_all("parkingCongestionDegree")[idx].text}")
        print(f"    - [입고된 차량 수] : {rs_xml.find_all("parkingOccupiedSpace")[idx].text}")
        temp = int(rs_xml.find_all("parkingTotalSpace")[idx].text) - int(rs_xml.find_all("parkingOccupiedSpace")[idx].text)
        print(f"    - [주차 가능 여유공간] : {temp}")
        print("")

-**--******- -******- -******- -******- -**-

[공항명] : 김포국제공항
[기준년월시] : 2025-02-21 10:13:03

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국내선 제1주차장
    - [총주차면수] : 2279
    - [주차장 혼잡정도] : 혼잡
    - [주차장 혼잡률] : 91.14%
    - [입고된 차량 수] : 2077
    - [전체 주차면 수] : 2279
    - [주차 가능 여유공간] : 202

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국내선 제2주차장
    - [총주차면수] : 1733
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 51.7%
    - [입고된 차량 수] : 896
    - [전체 주차면 수] : 1733
    - [주차 가능 여유공간] : 837

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국제선 주차빌딩
    - [총주차면수] : 567
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 89.77%
    - [입고된 차량 수] : 509
    - [전체 주차면 수] : 567
    - [주차 가능 여유공간] : 58

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 국제선 지하
    - [총주차면수] : 599
    - [주차장 혼잡정도] : 원활
    - [주차장 혼잡률] : 86.48%
    - [입고된 차량 수] : 518
    - [전체 주차면 수] : 599
    - [주차 가능 여유공간] : 81

-**- -**- -**- -**- -**- -**- -**- -**- -**-

[주차장명] : 화물청사
    - [총주차면수] : 737
    - [주차장 혼잡정

#### 수집한 데이터를 데이터 프레임에 저장하기
- 데이터 프레임 변수명 : df_all
- 컬럼명 : 공항명, 기준년월일, 기준시분초, 주차장명, 혼잡도, 혼잡율, 총주차면수, 입고된차량수
- 위 컬럼 외 추가하고 싶은 경우 자유롭게

In [29]:
# 빈 데이터프레임 생성
df_all = pd.DataFrame(columns=["공항명", "기준년월일", "기준시분초", "주차장명", "혼잡도", "혼잡율", "총주차면수", "입고된차량수", "주차여유공간"])

# 데이터 리스트 생성
data_list = []

# 공항 코드 리스트 순회
for num in range(len(schAirportCode_list)):
    api_url = f"http://openapi.airport.co.kr/service/rest/AirportParkingCongestion/airportParkingCongestionRT?schAirportCode={schAirportCode_list[num]}&serviceKey={api_key}&numOfRows={numOfRows}&pageNo={pageNo}"
    rs_data = requests.get(api_url)
    rs_text = rs_data.text
    rs_xml = BeautifulSoup(rs_text, "xml")

    airportKor = rs_xml.find("airportKor").text # 공항명
    sysGetdate = rs_xml.find("sysGetdate").text # 기준일
    sysGettime = rs_xml.find("sysGettime").text # 기준시

    # XML에서 모든 주차장 정보를 추출하여 데이터프레임에 추가
    for idx in range(len(rs_xml.find_all("airportKor"))):
        parkingAirportCodeName = rs_xml.find_all("parkingAirportCodeName")[idx].text # 주차장면
        parkingCongestion = rs_xml.find_all("parkingCongestion")[idx].text # 혼잡도
        parkingCongestionDegree = rs_xml.find_all("parkingCongestionDegree")[idx].text # 혼잡률
        parkingTotalSpace = int(rs_xml.find_all("parkingTotalSpace")[idx].text) # 총주차면수
        parkingOccupiedSpace = int(rs_xml.find_all("parkingOccupiedSpace")[idx].text) # 입고된 차량수

        # 주차 가능 여유공간 계산
        available_space = parkingTotalSpace - parkingOccupiedSpace

        # 데이터를 리스트에 추가
        data_list.append({
            "공항명" : airportKor,
            "기준년월일" : sysGetdate,
            "기준시분초" : sysGettime,
            "주차장명" : parkingAirportCodeName,
            "혼잡도" : parkingCongestion,
            "혼잡율" : parkingCongestionDegree,
            "총주차면수":  parkingTotalSpace,
            "입고된차량수" : parkingOccupiedSpace,
            "주차여유공간" : available_space
        })

# 리스트를 데이터프레임으로 변환
df_all = pd.DataFrame(data_list)

# 결과 출력
df_all

,공항명,기준년월일,기준시분초,주차장명,혼잡도,혼잡율,총주차면수,입고된차량수,주차여유공간
0,김포국제공항,2025-02-21,10:43:03,국내선 제1주차장,혼잡,91.84%,2279,2093,186
1,김포국제공항,2025-02-21,10:43:03,국내선 제2주차장,원활,51.24%,1733,888,845
2,김포국제공항,2025-02-21,10:43:03,국제선 주차빌딩,혼잡,91.89%,567,521,46
3,김포국제공항,2025-02-21,10:43:03,국제선 지하,원활,88.81%,599,532,67
4,김포국제공항,2025-02-21,10:43:03,화물청사,만차,100%,737,737,0
5,김해국제공항,2025-02-21,10:43:03,P1 여객주차장,만차,100%,2005,2005,0
6,김해국제공항,2025-02-21,10:43:03,P2 여객주차장,만차,100%,2453,2453,0
7,김해국제공항,2025-02-21,10:43:03,P3 여객(화물),만차,100%,878,878,0
8,제주국제공항,2025-02-21,10:43:03,P1주차장,만차,100%,1763,1763,0
9,제주국제공항,2025-02-21,10:43:03,P2장기주차장,원활,89.14%,488,435,53


In [30]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   공항명     22 non-null     object
 1   기준년월일   22 non-null     object
 2   기준시분초   22 non-null     object
 3   주차장명    22 non-null     object
 4   혼잡도     22 non-null     object
 5   혼잡율     22 non-null     object
 6   총주차면수   22 non-null     int64 
 7   입고된차량수  22 non-null     int64 
 8   주차여유공간  22 non-null     int64 
dtypes: int64(3), object(6)
memory usage: 1.7+ KB
